In [1]:
import pandas as pd
import librosa
import numpy as np
from tensorflow.keras import layers,models
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import save_model


In [2]:
csv_file_path = '/Users/dheemankumar/github/audio-ai/broken_3s_audio_data.csv'
df = pd.read_csv(csv_file_path)

In [3]:
audio_data = []
labels_noise=[]

# Step 2 and 3: Load audio files and process the data with a sample rate of 22050
for index, row in df.iterrows():
    audio_file_path = '/Users/dheemankumar/github/audio-ai/3sec_audio/' + row['name']  # Adjust the path as needed
    audio, sample_rate = librosa.load(audio_file_path, sr=22050)  # Load audio with a sample rate of 22050

    # Perform additional processing if needed, e.g., creating spectrograms\

    d=librosa.stft(audio)
    s_db=librosa.amplitude_to_db(np.abs(d),ref=np.max)

    s_db_with_channel = np.expand_dims(s_db, axis=-1)

    #print(s_db.shape)


    # Append the processed audio data and label to the lists
    audio_data.append(s_db_with_channel)
    labels_noise.append(row[["noise"]])

In [4]:
# Step 4: Create NumPy arrays
audio_data = np.array(audio_data)
labels_noise = np.array(labels_noise)

In [5]:
labels_noise.shape

(13388, 1)

In [6]:
audio_data.shape

(13388, 1025, 130, 1)

In [7]:
X_train_n, X_test_n, y_train_n, y_test_n = train_test_split(audio_data, labels_noise, test_size=0.2, random_state=42)

In [8]:
y_train_n[0]

array([0], dtype=object)

In [9]:
# Create a Sequential model_noise
model_noise = models.Sequential()

# Add a 2D convolutional layer with 32 filters, a 3x3 kernel, and 'relu' activation
model_noise.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(1025, 130, 1)))

# Add a max-pooling layer
model_noise.add(layers.MaxPooling2D((2, 2)))

# Add another 2D convolutional layer with 64 filters and 'relu' activation
model_noise.add(layers.Conv2D(64, (3, 3), activation='relu'))

# Add another max-pooling layer
model_noise.add(layers.MaxPooling2D((2, 2)))

# Add a flattening layer to convert to 1D tensor
model_noise.add(layers.Flatten())

# Add a fully connected (dense) layer with 64 units and 'relu' activation
model_noise.add(layers.Dense(64, activation='relu'))

# Add the output layer with 7 units (since you want 7 outputs) and 'softmax' activation
model_noise.add(layers.Dense(1, activation='sigmoid'))

# Compile the model_noise
model_noise.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Display the model_noise summary
model_noise.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 1023, 128, 32)     320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 511, 64, 32)       0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 509, 62, 64)       18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 254, 31, 64)       0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 503936)            0         
                                                                 
 dense (Dense)               (None, 64)                3

In [12]:
model_noise_2 = models.Sequential()

model_noise_2.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(128, 130, 1)))
model_noise_2.add(layers.BatchNormalization())
model_noise_2.add(layers.MaxPooling2D((2, 2)))

model_noise_2.add(layers.Conv2D(128, (3, 3), activation='relu'))
model_noise_2.add(layers.BatchNormalization())
model_noise_2.add(layers.MaxPooling2D((2, 2)))

model_noise_2.add(layers.Conv2D(256, (3, 3), activation='relu'))
model_noise_2.add(layers.BatchNormalization())
model_noise_2.add(layers.MaxPooling2D((2, 2)))

model_noise_2.add(layers.Flatten())

model_noise_2.add(layers.Dense(128, activation='relu'))
model_noise_2.add(layers.BatchNormalization())
model_noise_2.add(layers.Dense(64, activation='relu'))
model_noise_2.add(layers.Dense(1, activation='softmax'))

model_noise_2.compile(optimizer='adam',
                       loss='cbinary_crossentropy',
                       metrics=['accuracy'])

model_noise_2.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 126, 128, 64)      640       
                                                                 
 batch_normalization_8 (Bat  (None, 126, 128, 64)      256       
 chNormalization)                                                
                                                                 
 max_pooling2d_8 (MaxPoolin  (None, 63, 64, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_9 (Conv2D)           (None, 61, 62, 128)       73856     
                                                                 
 batch_normalization_9 (Bat  (None, 61, 62, 128)       512       
 chNormalization)                                                
                                                      

In [13]:
# Convert data types if needed
X_train_n = np.asarray(X_train_n, dtype=np.float32)
X_test_n = np.asarray(X_test_n, dtype=np.float32)
y_train_n = np.asarray(y_train_n, dtype=np.float32)
y_test_n = np.asarray(y_test_n, dtype=np.float32)

In [15]:
ep=1
history_noise = model_noise.fit(X_train_n, y_train_n, epochs=ep, validation_data=(X_test_n, y_test_n), batch_size=64)

KeyboardInterrupt: 

In [ ]:
model_noise.save("models/NoiseModel_2.h5")

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
import json

# Save history to a JSON file
with open('history_noise_2.json', 'w') as json_file:
    json.dump(history_noise.history, json_file)